# TP FINAL BIF DATA

Entrega final del trabajo práctico

In [ ]:
# Bloque 1: Configuración y Carga
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count, mode, mean
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

# Ruta al archivo (Asegúrate de subirlo a Databricks mediante 'File -> Upload Data')
# Una vez subido, Databricks te dará la ruta, suele ser algo como:
file_path = "/FileStore/tables/heart_disease.csv" 

# Cargar CSV
df = spark.read.csv(file_path, header=True, inferSchema=True)

print("Datos originales:")
df.show(5)

# Bloque 2: Limpieza y Preprocesamiento (Data Engineering)
# 1. Tratar valores nulos (He detectado nulos en 'ca' y 'thal' en tu archivo)
# Vamos a rellenarlos con la moda (el valor más común) o un valor fijo.
# Para simplificar en este TP, usaremos -1 para indicar que faltaba el dato, o la moda.
# Aquí rellenamos con la media para 'ca' y la moda para 'thal' (simplificado para Spark):
df_clean = df.na.fill({'ca': 0.0, 'thal': 3.0}) # Asumimos valores comunes

# 2. Transformar la variable objetivo 'num'
# En este dataset, 0 es sano, y 1,2,3,4 son distintos grados de enfermedad.
# Para el TP, conviene hacerlo binario: 0 = No Enfermo, 1 = Enfermo.
df_final = df_clean.withColumn("label", when(col("num") > 0, 1).otherwise(0))

# 3. Separar Features Numéricas y Categóricas
# Esto es clave para el pipeline de ML posterior
categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']
numerical_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'ca']

# (Opcional) Puedes castear categóricas a String si quieres usar OneHotEncoder después
# Por ahora las dejamos numéricas para un modelo simple (Random Forest las maneja bien)

# Bloque 3: Split y Guardado
# Dividimos en Train (80%) y Test (20%)
train_data, test_data = df_final.randomSplit([0.8, 0.2], seed=42)

# Guardar en formato DELTA (Mejor práctica en Databricks)
train_data.write.format("delta").mode("overwrite").saveAsTable("heart_train")
test_data.write.format("delta").mode("overwrite").saveAsTable("heart_test")

print(f"Datos de entrenamiento guardados: {train_data.count()} registros.")
print(f"Datos de test guardados: {test_data.count()} registros.")

Verificando e instalando dependencias...

✓ pandas ya está instalado
✓ numpy ya está instalado
✓ matplotlib ya está instalado
✓ seaborn ya está instalado
✓ requests ya está instalado
✓ requests_cache ya está instalado

✓ Todas las dependencias están listas

